## 下载数据集

In [3]:
# windows电脑自己手动下载即可
# 数据集网址，自己解压至E:\mmpose\data目录下即可
!wget https://zihao-download.obs.cn-east-3.myhuaweicloud.com/yolov8/datasets/Triangle_215_Dataset/Triangle_215_Keypoint_coco.zip

# 下载labelme格式数据集(这个不全，仅作实验使用)
!wget https://zihao-download.obs.cn-east-3.myhuaweicloud.com/yolov8/datasets/SJB_25_Dataset.zip

'wget' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


## 下载测试数据

In [4]:
# 这个目录存放测试集
!mkdir data/test_triangle

命令语法不正确。


In [ ]:
# windows电脑自己手动下载即可
# 图像-30度直角三角板，拍摄：同济子豪兄、田文博
!wget https://zihao-openmmlab.obs.myhuaweicloud.com/20220610-mmpose/triangle_dataset/test_img/triangle_1.jpg -P data/test_triangle
!wget https://zihao-openmmlab.obs.myhuaweicloud.com/20220610-mmpose/triangle_dataset/test_img/triangle_2.jpg -P data/test_triangle
!wget https://zihao-openmmlab.obs.myhuaweicloud.com/20220610-mmpose/triangle_dataset/test_img/triangle_3.jpg -P data/test_triangle
!wget https://zihao-openmmlab.obs.myhuaweicloud.com/20220610-mmpose/triangle_dataset/test_img/triangle_4.jpg -P data/test_triangle

# 视频-30度直角三角板，拍摄：同济子豪兄，田文博
!wget https://zihao-openmmlab.obs.myhuaweicloud.com/20220610-mmpose/triangle_dataset/videos/triangle_6.mp4 -P data/test_triangle
!wget https://zihao-openmmlab.obs.myhuaweicloud.com/20220610-mmpose/triangle_dataset/videos/triangle_7.mp4 -P data/test_triangle
!wget https://zihao-openmmlab.obs.myhuaweicloud.com/20220610-mmpose/triangle_dataset/videos/triangle_9.mp4 -P data/test_triangle

## 随便看一张数据集中的图片

In [ ]:
from PIL import Image
Image.open('E:\mmpose\data\Triangle_215_Keypoint_coco\images\DSC_0373.jpg')

## 划分训练集、测试集

In [1]:
import os
import shutil
import random

from tqdm import tqdm

test_frac = 0.2  # 测试集比例
random.seed(123) # 随机数种子，便于复现

folder = 'E:\mmpose\data\Triangle_215_Keypoint_labelme\labelme_jsons'

img_paths = os.listdir(folder)
# print(img_paths)
random.shuffle(img_paths) # 随机打乱

val_number = int(len(img_paths) * test_frac) # 测试集文件个数
train_files = img_paths[val_number:]         # 训练集文件名列表
val_files = img_paths[:val_number]           # 测试集文件名列表

print('数据集文件总数', len(img_paths))
print('训练集文件个数', len(train_files))
print('测试集文件个数', len(val_files))

数据集文件总数 25
训练集文件个数 20
测试集文件个数 5


## 将训练集json文件移动到train_labelme_jsons文件夹

In [12]:
# 创建文件夹，存放训练集的 labelme格式的 json 标注文件
# Triangle_215_Keypoint_labelme目录下一共2个文件夹，1个是images装原图，1个是labelme_jsons装.json
# 这里将labelme_jsons中所有.json分成2类，分别创建两个文件夹装这两类文件
train_labelme_jsons_folder = 'E:\\mmpose\\data\\Triangle_215_Keypoint_labelme\\labelme_jsons\\train_labelme_jsons'
os.mkdir(train_labelme_jsons_folder)

for each in tqdm(train_files):
    src_path = os.path.join(folder, each)
    dst_path = os.path.join(train_labelme_jsons_folder, each)
    shutil.move(src_path, dst_path)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<?, ?it/s]


In [5]:
len(os.listdir(train_labelme_jsons_folder))

20

## 将测试集json文件移动到val_labelme_jsons文件夹

In [13]:
# 创建文件夹，存放测试集的 labelme格式的 json 标注文件
val_labelme_jsons_folder = 'E:\\mmpose\\data\\Triangle_215_Keypoint_labelme\\labelme_jsons\\val_labelme_jsons'
os.mkdir(val_labelme_jsons_folder)

for each in tqdm(val_files):
    src_path = os.path.join(folder, each)
    dst_path = os.path.join(val_labelme_jsons_folder, each)
    shutil.move(src_path, dst_path)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<?, ?it/s]


In [14]:
len(os.listdir(val_labelme_jsons_folder))

5

## 指定数据集信息

In [7]:
Dataset_root = 'E:\\mmpose\\data\\Triangle_215_Keypoint_labelme'

class_list = {
    'supercategory': 'sjb_rect',
    'id': 1,
    'name': 'sjb_rect',
    'keypoints': ['angle_30', 'angle_60', 'angle_90'], # 大小写敏感
    'skeleton':[[0,1], [0,2], [1,2]]
}

## 函数-处理单个labelme标注json文件

In [9]:
def process_single_json(labelme, image_id=1):
    '''
    输入labelme的json数据，输出coco格式的每个框的关键点标注信息
    '''
    
    global ANN_ID
    
    coco_annotations = []
    
    for each_ann in labelme['shapes']: # 遍历该json文件中的所有标注

        if each_ann['shape_type'] == 'rectangle': # 筛选出个体框

            # 个体框元数据
            bbox_dict = {}
            bbox_dict['category_id'] = 1
            bbox_dict['segmentation'] = []
            
            bbox_dict['iscrowd'] = 0
            bbox_dict['segmentation'] = []
            bbox_dict['image_id'] = image_id
            bbox_dict['id'] = ANN_ID
            # print(ANN_ID)
            ANN_ID += 1

            # 获取个体框坐标
            bbox_left_top_x = min(int(each_ann['points'][0][0]), int(each_ann['points'][1][0]))
            bbox_left_top_y = min(int(each_ann['points'][0][1]), int(each_ann['points'][1][1]))
            bbox_right_bottom_x = max(int(each_ann['points'][0][0]), int(each_ann['points'][1][0]))
            bbox_right_bottom_y = max(int(each_ann['points'][0][1]), int(each_ann['points'][1][1]))
            bbox_w = bbox_right_bottom_x - bbox_left_top_x
            bbox_h = bbox_right_bottom_y - bbox_left_top_y
            bbox_dict['bbox'] = [bbox_left_top_x, bbox_left_top_y, bbox_w, bbox_h] # 左上角x、y、框的w、h
            bbox_dict['area'] = bbox_w * bbox_h
            
            # 筛选出分割多段线
            for each_ann in labelme['shapes']: # 遍历所有标注
                if each_ann['shape_type'] == 'polygon': # 筛选出分割多段线标注
                    # 第一个点的坐标
                    first_x = each_ann['points'][0][0]
                    first_y = each_ann['points'][0][1]
                    if (first_x>bbox_left_top_x) & (first_x<bbox_right_bottom_x) & (first_y<bbox_right_bottom_y) & (first_y>bbox_left_top_y): # 筛选出在该个体框中的关键点
                        bbox_dict['segmentation'] = list(map(lambda x: list(map(lambda y: round(y, 2), x)), each_ann['points'])) # 坐标保留两位小数
                        # bbox_dict['segmentation'] = each_ann['points']

            # 筛选出该个体框中的所有关键点
            bbox_keypoints_dict = {}
            for each_ann in labelme['shapes']: # 遍历所有标注
                
                if each_ann['shape_type'] == 'point': # 筛选出关键点标注
                    # 关键点横纵坐标
                    x = int(each_ann['points'][0][0])
                    y = int(each_ann['points'][0][1])
                    label = each_ann['label']
                    if (x>bbox_left_top_x) & (x<bbox_right_bottom_x) & (y<bbox_right_bottom_y) & (y>bbox_left_top_y): # 筛选出在该个体框中的关键点
                        bbox_keypoints_dict[label] = [x, y]
                        
            bbox_dict['num_keypoints'] = len(bbox_keypoints_dict)
            # print(bbox_keypoints_dict)

            # 把关键点按照类别顺序排好
            bbox_dict['keypoints'] = []
            for each_class in class_list['keypoints']:
                if each_class in bbox_keypoints_dict:
                    bbox_dict['keypoints'].append(bbox_keypoints_dict[each_class][0])
                    bbox_dict['keypoints'].append(bbox_keypoints_dict[each_class][1])
                    bbox_dict['keypoints'].append(2) # 2-可见不遮挡 1-遮挡 0-没有点
                else: # 不存在的点，一律为0
                    bbox_dict['keypoints'].append(0)
                    bbox_dict['keypoints'].append(0)
                    bbox_dict['keypoints'].append(0)
                    
            coco_annotations.append(bbox_dict)
            
    return coco_annotations

## 函数-转换当前目录下所有labelme格式的json文件

In [10]:
def process_folder():
    IMG_ID = 0
    ANN_ID = 0

    # 遍历所有 labelme 格式的 json 文件
    for labelme_json in os.listdir(): 

        if labelme_json.split('.')[-1] == 'json':

            with open(labelme_json, 'r', encoding='utf-8') as f:

                labelme = json.load(f)

                ## 提取图像元数据
                img_dict = {}
                img_dict['file_name'] = labelme['imagePath']
                img_dict['height'] = labelme['imageHeight']
                img_dict['width'] = labelme['imageWidth']
                img_dict['id'] = IMG_ID
                coco['images'].append(img_dict)

                ## 提取框和关键点信息
                coco_annotations = process_single_json(labelme, image_id=IMG_ID)
                coco['annotations'] += coco_annotations

                IMG_ID += 1

                print(labelme_json, '已处理完毕')

        else:
            pass

## 转换训练集的所有labelme标注文件

In [12]:
import os
import json
import numpy as np

coco = {} # 暂时存放所有json文件整合后的coco格式标签信息

coco['categories'] = []
coco['categories'].append(class_list)

coco['images'] = []
coco['annotations'] = []

IMG_ID = 0
ANN_ID = 0

path = os.path.join(Dataset_root, 'labelme_jsons', 'train_labelme_jsons')
os.chdir(path)

In [ ]:
process_folder()

In [14]:
# 保存coco标注文件
coco_path = '../../train_coco.json'
with open(coco_path, 'w') as f:
    json.dump(coco, f, indent=2)
    
os.chdir('../../')

## 验证训练集的MS COCO格式标注

In [19]:
os.listdir()

['images', 'labelme_jsons', 'train_coco.json']

In [17]:
from pycocotools.coco import COCO
my_coco = COCO('train_coco.json')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


## 转换测试集的所有labelme标注文件

In [20]:
coco = {}

coco['categories'] = []
coco['categories'].append(class_list)

coco['images'] = []
coco['annotations'] = []

IMG_ID = 0
ANN_ID = 0

path = os.path.join('labelme_jsons', 'val_labelme_jsons')
os.chdir(path)

In [21]:
process_folder()

DSC_0245.json 已处理完毕
DSC_0260.json 已处理完毕
DSC_0265.json 已处理完毕
DSC_0274.json 已处理完毕
DSC_0280.json 已处理完毕


In [22]:
# 保存coco标注文件
coco_path = '../../val_coco.json'
with open(coco_path, 'w') as f:
    json.dump(coco, f, indent=2)
    
os.chdir('../../')

## 验证测试集的MS COCO格式标注

In [23]:
from pycocotools.coco import COCO
my_coco = COCO('val_coco.json')

# 之后记得手动删除之前旧的格式数据

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
